# SSM 계산 : 차량 페어, 롱타입
* Heatmap으로 나타내기
* 시공도로 나타내기

# Import

In [2]:
import pandas as pd
import numpy as np

from tqdm import tqdm
import warnings
import os

import math # arctangent; math.atan 사용 목적
import statistics

from SSM_base import *
from SSM_time_based import *
from SSM_deceleration_based import *
from SSM_energy_based import *

import pyarrow.parquet as pq
#df = pq.read_pandas('data.parquet').to_pandas()

In [3]:
#warnings.filterwarnings('ignore')

# Load Dataset

In [4]:
working_dir = 'D:/OneDrive/Projects/2023_SSM_Feasibility/Dataset'

In [5]:
folder_name = '02_processed_long'
file_name = 'merging_type1_03frm_0.10sec.parquet'
file_path = os.path.join(working_dir, folder_name, file_name)

In [6]:
df = pq.read_pandas(file_path).to_pandas()

df.rename({'LV_V_len' : 'LV_len',
           'LV_V_wid' : 'LV_wid'}, axis = 1, inplace = True)

In [7]:
df.head(3)

,pair,veh_id,LV_ID,frm,LV_type,local_x,local_y,V_len,V_wid,velocity,...,LV_Lane_00,LV_Lane_99,LV_Lane_change,LV_Lane_leave,LV_Lane_change_direction,LV_LC_CF,D_x,D_y,D,D_gap
0,1000_994,1000,994,17469,LVL,181.875,21.56250,7.09375,2.064453,104.375,...,U3,U3,None,None,Straight,CF,21.125,-7.820312,22.526051,-0.461596
1,1000_994,1000,994,17463,LVL,176.125,21.46875,7.09375,2.064453,104.000,...,U3,U3,None,None,Straight,CF,22.250,-7.773438,23.568811,-0.701797
2,1000_994,1000,994,17460,LVL,173.125,21.40625,7.09375,2.064453,104.125,...,U3,U3,None,None,Straight,CF,23.000,-7.750000,24.270609,-0.593082


In [8]:
list(df.columns)

['pair',
 'veh_id',
 'LV_ID',
 'frm',
 'LV_type',
 'local_x',
 'local_y',
 'V_len',
 'V_wid',
 'velocity',
 'acc',
 'lane',
 'Time',
 'local_x_before',
 'local_y_before',
 'delta_local_x',
 'delta_local_y',
 'velocity_x',
 'velocity_y',
 'velocity_x_before',
 'velocity_y_before',
 'delta_velocity_x',
 'delta_velocity_y',
 'acc_x',
 'acc_y',
 'lane_past',
 'Lane_record',
 'Lane_record_split',
 'Lane_00',
 'Lane_99',
 'Lane_change',
 'Lane_leave',
 'Lane_change_direction',
 'LC_CF',
 'LV_local_x',
 'LV_local_y',
 'LV_len',
 'LV_wid',
 'LV_velocity',
 'LV_acc',
 'LV_lane',
 'LV_Time',
 'LV_local_x_before',
 'LV_local_y_before',
 'LV_delta_local_x',
 'LV_delta_local_y',
 'LV_velocity_x',
 'LV_velocity_y',
 'LV_velocity_x_before',
 'LV_velocity_y_before',
 'LV_delta_velocity_x',
 'LV_delta_velocity_y',
 'LV_acc_x',
 'LV_acc_y',
 'LV_lane_past',
 'LV_Lane_record',
 'LV_Lane_record_split',
 'LV_Lane_00',
 'LV_Lane_99',
 'LV_Lane_change',
 'LV_Lane_leave',
 'LV_Lane_change_direction',
 'LV_LC_

# Calculation

## Settings
* 공통 상수 설정

In [28]:
threshold_TTC = 3 # time-based SSM 임계값
threshold_DRAC = 3.4
threshold_PET = 1

tmsec = 0.1 # 각 row별 시간단위(초)
g = 9.81 # 중력가속도
f = 0.35 # 마찰계수. friction coeefficient
delta_t = 1 # Reaction Delay time. 운전자 반응시간
MADR = 8.45 # 차량의 최대 감속 능력. 날씨에 따라, 차종에 따라 다르다. Saccomanno et al.(2008) 참조
deceleration_rate = 0.3 * g # 차량 감속도. 3.3도 됨

## Time-based SSM
* Temporal Proximal Indicator

In [10]:
tqdm.pandas()

In [11]:
# 꼭지점 4개 만들기
# 각 차량의 1,2,3,4번 꼭지점 좌표를 구한다.
df['V_point_1'], df['V_point_2'], df['V_point_3'], df['V_point_4'] = zip(*df.progress_apply(lambda x: points(x.local_x, x.local_y, x.velocity_x, x.velocity_y, x.V_wid, x.V_len), axis = 1))
df['LV_point_1'], df['LV_point_2'], df['LV_point_3'], df['LV_point_4'] = zip(*df.progress_apply(lambda x: points(x.LV_local_x, x.LV_local_y, x.LV_velocity_x, x.LV_velocity_y, x.LV_wid, x.LV_len), axis = 1))

100%|███████████████████████████████████████████████████████████████████████| 156096/156096 [00:08<00:00, 18359.83it/s]


In [12]:
## Nearest_point 추출
df['V_LV_point_num'], df['V_LV_point_n'], df['LV_point_n'], df['V_LV_point_num2'], df['V_LV_point_n2'], df['LV_point_n2'] = zip(*df.progress_apply(lambda x: nearest_point(x.V_point_1, x.V_point_2, x.V_point_3, x.V_point_4, x.LV_point_1, x.LV_point_2, x.LV_point_3, x.LV_point_4), axis = 1))

100%|███████████████████████████████████████████████████████████████████████| 156096/156096 [00:15<00:00, 10120.17it/s]


In [13]:
## Shortest Distance 구하기 : ACT 용
df['V_LV_point_delta'], df['V_LV_shortest_vector'], df['V_LV_shortest_distance'] = zip(*df.progress_apply(lambda x: shortest_distance(
    x.V_LV_point_num, x.V_LV_point_n, x.LV_point_n, 
    x.V_point_1, x.V_point_2, x.V_point_3, x.V_point_4, 
    x.LV_point_1, x.LV_point_2, x.LV_point_3, x.LV_point_4), axis = 1))

100%|███████████████████████████████████████████████████████████████████████| 156096/156096 [00:14<00:00, 10776.20it/s]


In [14]:
df['overlap'] = df.progress_apply(lambda x: overlap(x.LV_type, x.V_LV_point_num, x.V_LV_point_num2, x.velocity_x, x.velocity_y, x.LV_velocity_x, x.LV_velocity_y, x.local_x, x.local_y, x.LV_local_x, x.LV_local_y, x.V_len, x.LV_len), axis = 1)

df['potential_conflict_type'], df['degX'] = zip(*df.progress_apply(lambda x: potential_conflict_type(x.LV_type, x.overlap, x.V_LV_point_num, x.V_LV_point_num2, x.velocity_x, x.velocity_y, x.LV_velocity_x, x.LV_velocity_y, x.local_x, x.local_y, x.LV_local_x, x.LV_local_y, x.V_len, x.V_wid, x.LV_len, x.LV_wid), axis = 1))

100%|████████████████████████████████████████████████████████████████████████| 156096/156096 [00:16<00:00, 9223.16it/s]


In [15]:
# 2차원 SSM
df['T2'] = df.progress_apply(lambda x: T2(
    x.V_LV_point_num, x.V_LV_point_n, x.LV_point_n, 
    x.V_point_1, x.V_point_2, x.V_point_3, x.V_point_4, 
    x.LV_point_1, x.LV_point_2, x.LV_point_3, x.LV_point_4,
    x.velocity_x, x.velocity_y, x.LV_velocity_x, x.LV_velocity_y), axis = 1)

100%|████████████████████████████████████████████████████████████████████████| 156096/156096 [00:39<00:00, 3993.31it/s]


In [16]:
#df[['ip1', 'ip2', 'cos_theta', 'theta', 'degX', 'potential_conflict_type']]

In [17]:
df['TTC'] = df.progress_apply(lambda x: TTC(x.potential_conflict_type, x.LV_type, x.local_x, x.LV_local_x, x.velocity_x, x.LV_velocity_x, x.V_len, x.LV_len, x.V_wid, x.LV_wid, x.local_y, x.LV_local_y, x.velocity_y, x.LV_velocity_y), axis = 1)

df['MTTC'] = df.progress_apply(lambda x: MTTC(x.potential_conflict_type, x.LV_type, x.local_x, x.LV_local_x, x.velocity_x, x.LV_velocity_x, x.acc_x, x.LV_acc_x, x.V_len, x.LV_len, x.V_wid, x.LV_wid, x.local_y, x.LV_local_y, x.velocity_y, x.LV_velocity_y), axis = 1)

df['CI_MTTC'] = df.progress_apply(lambda x: CI_MTTC(x.local_x, x.LV_local_x, x.velocity, x.LV_velocity, x.acc_x, x.LV_acc_x, x.MTTC), axis = 1)

100%|███████████████████████████████████████████████████████████████████████| 156096/156096 [00:08<00:00, 18986.70it/s]


In [18]:
df.columns

Index(['pair', 'veh_id', 'LV_ID', 'frm', 'LV_type', 'local_x', 'local_y',
       'V_len', 'V_wid', 'velocity', 'acc', 'lane', 'Time', 'local_x_before',
       'local_y_before', 'delta_local_x', 'delta_local_y', 'velocity_x',
       'velocity_y', 'velocity_x_before', 'velocity_y_before',
       'delta_velocity_x', 'delta_velocity_y', 'acc_x', 'acc_y', 'lane_past',
       'Lane_record', 'Lane_record_split', 'Lane_00', 'Lane_99', 'Lane_change',
       'Lane_leave', 'Lane_change_direction', 'LC_CF', 'LV_local_x',
       'LV_local_y', 'LV_len', 'LV_wid', 'LV_velocity', 'LV_acc', 'LV_lane',
       'LV_Time', 'LV_local_x_before', 'LV_local_y_before', 'LV_delta_local_x',
       'LV_delta_local_y', 'LV_velocity_x', 'LV_velocity_y',
       'LV_velocity_x_before', 'LV_velocity_y_before', 'LV_delta_velocity_x',
       'LV_delta_velocity_y', 'LV_acc_x', 'LV_acc_y', 'LV_lane_past',
       'LV_Lane_record', 'LV_Lane_record_split', 'LV_Lane_00', 'LV_Lane_99',
       'LV_Lane_change', 'LV_Lane_leave', 

In [19]:
df['ACT'], df['total_rel'], df['d_delta'] = zip(*df.progress_apply(lambda x: ACT(
    x.velocity_x, x.velocity_y, x.velocity_x_before, x.velocity_y_before,
    x.LV_velocity_x, x.LV_velocity_y, x.LV_velocity_x_before, x.LV_velocity_y_before,
    x.acc_x, x.acc_y, x.LV_acc_x, x.LV_acc_y, 
    x.V_LV_shortest_vector, x.V_LV_shortest_distance), axis = 1))

100%|████████████████████████████████████████████████████████████████████████| 156096/156096 [00:18<00:00, 8236.39it/s]


In [20]:
df['pPET'], df['time_gap'] = zip(*df.progress_apply(lambda x: pPET(
    x.potential_conflict_type, x.V_LV_point_num, x.V_LV_point_n, x.LV_point_n, 
    x.V_point_1, x.V_point_2, x.V_point_3, x.V_point_4, 
    x.LV_point_1, x.LV_point_2, x.LV_point_3, x.LV_point_4,
    x.velocity_x, x.velocity_y, x.LV_velocity_x, x.LV_velocity_y), axis = 1))

100%|████████████████████████████████████████████████████████████████████████| 156096/156096 [02:24<00:00, 1080.95it/s]


In [21]:
#df['TIT2'] = df.progress_apply(lambda x: TIT(x.T2, threshold_TTC, tmsec), axis = 1)
#df['TIACT'] = df.progress_apply(lambda x: TIT(x.ACT, threshold_TTC, tmsec), axis = 1)

In [22]:
# df['SSCR_TTC_prime'] = df.progress_apply(lambda x: SSCR_TTC_prime(
#     x.potential_conflict_type, x.V_LV_point_num, x.V_LV_point_n, x.LV_point_n, 
#     x.V_point_1, x.V_point_2, x.V_point_3, x.V_point_4, 
#     x.LV_point_1, x.LV_point_2, x.LV_point_3, x.LV_point_4,
#     x.velocity_x, x.velocity_y, x.LV_velocity_x, x.LV_velocity_y), axis = 1)

In [23]:
# df['SSCR_R'] = df.progress_apply(lambda x: SSCR_R(x.TTC), axis = 1)
# df['SSCR_Y'] = df.progress_apply(lambda x: SSCR_Y(x.TTC, x.pPET, x.SSCR_TTC_prime), axis = 1)
# df['SSCR'] = df['SSCR_R'] + df['SSCR_Y']

# # df = TA(df) # Time to Accident 계산
# # df['critical_speed'] = df.apply(lambda x: critical_speed(g, f, x.pPET), axis = 1)
# # df['TA_CS'] = df.apply(lambda x: TA_CS(x.TA, x.critical_speed), axis = 1) # TA/CS 계산

## Deceleration-based SSM

In [24]:
df['PSD'] = df.progress_apply(lambda x: PSD(x.potential_conflict_type, x.LV_type, x.local_x, x.LV_local_x, x.velocity_x, x.LV_velocity_x, x.V_len, x.LV_len, f, g, x.V_wid, x.LV_wid, x.local_y, x.LV_local_y, x.velocity_y, x.LV_velocity_y), axis = 1)

df['DSS'] = df.progress_apply(lambda x: DSS(x.potential_conflict_type, x.LV_type, x.local_x, x.LV_local_x, x.velocity_x, x.LV_velocity_x, x.V_len, x.LV_len, g, f, delta_t, x.V_wid, x.LV_wid, x.local_y, x.LV_local_y, x.velocity_y, x.LV_velocity_y), axis = 1)
#df['TIDSS'] = df.progress_apply(lambda x: TIDSS(x.DSS, tmsec), axis = 1)

df['PICUD'] = df.progress_apply(lambda x: PICUD(x.potential_conflict_type, x.LV_type, x.local_x, x.LV_local_x, x.velocity_x, x.LV_velocity_x, x.D_x, delta_t, deceleration_rate, x.V_wid, x.LV_wid, x.local_y, x.LV_local_y, x.velocity_y, x.LV_velocity_y), axis = 1)

df['MTC'] = df.progress_apply(lambda x: MTC(x.potential_conflict_type, x.LV_type, x.local_x, x.LV_local_x, x.D_x, x.V_len, x.LV_len, x.velocity_x, x.LV_velocity, delta_t, deceleration_rate, x.V_wid, x.LV_wid, x.local_y, x.LV_local_y, x.velocity_y, x.LV_velocity_y), axis = 1)
df['MMTC'] = df.progress_apply(lambda x: MMTC(x.potential_conflict_type, x.LV_type, x.local_x, x.LV_local_x, x.velocity_x, x.LV_velocity_x, g, x.V_wid, x.LV_wid, x.local_y, x.LV_local_y, x.velocity_y, x.LV_velocity_y), axis = 1)

#df['RCRI'] = df.progress_apply(lambda x: RCRI(x.local_x, x.LV_local_x, x.velocity_x, x.LV_velocity_x, x.acc_x, x.LV_acc_x, x.D_x, delta_t, x.V_wid, x.LV_wid, x.local_y, x.LV_local_y, x.velocity_y, x.LV_velocity_y), axis = 1)
#df['TERCRI'] = df.progress_apply(lambda x: TERCRI(x.RCRI, tmsec), axis = 1)

100%|███████████████████████████████████████████████████████████████████████| 156096/156096 [00:12<00:00, 12657.45it/s]


In [25]:
df['DRAC'] = df.progress_apply(lambda x: DRAC(x.potential_conflict_type, x.LV_type, x.local_x, x.LV_local_x, x.D_x, x.velocity_x, x.LV_velocity_x, x.LV_len, x.V_len, x.V_wid, x.LV_wid, x.local_y, x.LV_local_y, x.velocity_y, x.LV_velocity_y), axis = 1)
#df['TIDRAC'] = df.apply(lambda x: TIDRAC(x.DRAC, threshold_DRAC), axis = 1)

df['MDRAC'] = df.progress_apply(lambda x: MDRAC(x.potential_conflict_type, x.LV_type, x.local_x, x.LV_local_x, x.velocity_x, x.LV_velocity_x, x.TTC, delta_t, x.V_wid, x.LV_wid, x.local_y, x.LV_local_y, x.velocity_y, x.LV_velocity_y), axis = 1)

df['DCIA'] = df.progress_apply(lambda x: DCIA(x.potential_conflict_type, x.LV_type, x.local_x, x.LV_local_x, x.velocity_x, x.LV_velocity_x, x.acc_x, x.LV_acc_x, delta_t, x.D_x, x.V_wid, x.LV_wid, x.local_y, x.LV_local_y, x.velocity_y, x.LV_velocity_y), axis = 1)

df['unsafety'] = df.progress_apply(lambda x: unsafety(x.potential_conflict_type, x.LV_type, x.local_x, x.LV_local_x, x.velocity_x, x.LV_velocity_x, x.acc_x, x.LV_acc_x, deceleration_rate, x.V_wid, x.LV_wid, x.local_y, x.LV_local_y, x.velocity_y, x.LV_velocity_y), axis = 1)

100%|███████████████████████████████████████████████████████████████████████| 156096/156096 [00:14<00:00, 11091.24it/s]


In [29]:
df['TIT'] = df.progress_apply(lambda x: TIT(x.TTC, threshold_TTC, tmsec), axis = 1)
df['TIT2'] = df.progress_apply(lambda x: TIT(x.T2, threshold_TTC, tmsec), axis = 1)
df['TIACT'] = df.progress_apply(lambda x: TIT(x.ACT, threshold_TTC, tmsec), axis = 1)
df['TITAdv'] = df.progress_apply(lambda x: TIT(x.pPET, threshold_PET, tmsec), axis = 1)
df['TITimeGap'] = df.progress_apply(lambda x: TIT(x.time_gap, threshold_PET, tmsec), axis = 1)

df['TIPICUD'] = df.progress_apply(lambda x: TIDSS(x.PICUD, tmsec), axis = 1)
df['TIDSS'] = df.progress_apply(lambda x: TIDSS(x.DSS, tmsec), axis = 1)

df['TIDRAC'] = df.progress_apply(lambda x: TIDRAC(x.DRAC, threshold_DRAC), axis = 1)
df['TIMDRAC'] = df.progress_apply(lambda x: TIDRAC(x.MDRAC, threshold_DRAC), axis = 1)
df['TIDCIA'] = df.progress_apply(lambda x: TIDRAC(x.DCIA, threshold_DRAC), axis = 1)

100%|███████████████████████████████████████████████████████████████████████| 156096/156096 [00:02<00:00, 61075.44it/s]


## Energy-based SSM

In [ ]:
# df['CIF'] = df.progress_apply(lambda x: CIF(x.velocity_x, x.TTC), axis = 1)
# df['CIF2'] = df.progress_apply(lambda x: CIF(x.velocity, x.T2), axis = 1)
# df['ACIF'] = df.progress_apply(lambda x: CIF(x.velocity, x.ACT), axis = 1)

# Save

In [30]:
delete_cols = ['V_point_1', 'V_point_2', 'V_point_3', 'V_point_4', 
              'LV_point_1', 'LV_point_2', 'LV_point_3', 'LV_point_4',
              'V_LV_point_num', 'V_LV_point_n', 'LV_point_n', 'V_LV_point_num2',
              'V_LV_point_n2',	'LV_point_n2',	'V_LV_point_delta',	'V_LV_shortest_vector',
              'V_LV_shortest_distance', 'total_rel', 'd_delta']

df2 = df.copy()
df2.drop(delete_cols, inplace = True, axis = 1)

df2 = df2.dropna(subset = ['veh_id', 'frm'])

In [31]:
folder_name = '03_SSM'
save_name = file_name
save_path = os.path.join(working_dir, folder_name, save_name)

df2.to_parquet(save_path, engine = 'fastparquet', compression = 'gzip') 

In [32]:
df2[['pPET', 'T2', 'ACT']]

,pPET,T2,ACT
0,NaN,NaN,NaN
1,NaN,NaN,25.495695
2,166.272222,NaN,11.052962
3,16.158800,NaN,NaN
4,57.783751,NaN,5.480418
...,...,...,...
156091,NaN,NaN,17.434579
156092,NaN,NaN,2.892386
156093,0.730234,NaN,NaN
156094,10.915368,NaN,NaN


In [33]:
df[['V_LV_shortest_distance', 'total_rel', 'd_delta', 'T2', 'TTC', 'MTTC', 'ACT']]

,V_LV_shortest_distance,total_rel,d_delta,T2,TTC,MTTC,ACT
0,18.483160,"[-0.5787549145584039, -1.0627430805633216]",NaN,NaN,NaN,NaN,NaN
1,19.602317,"[0.7292974385078896, 0.24341870553161338]",0.768848,NaN,NaN,NaN,25.495695
2,20.340411,"[1.5242324659257946, 1.031165159223799]",1.840268,NaN,NaN,NaN,11.052962
3,20.985508,"[-0.35786780127052353, -0.8226833895673269]",NaN,NaN,NaN,NaN,NaN
4,21.504437,"[3.0196445763256996, 2.505690343207647]",3.923868,NaN,NaN,NaN,5.480418
...,...,...,...,...,...,...,...
156091,11.436682,"[0.45153428759704584, 0.4758388187929003]",0.655977,NaN,NaN,NaN,17.434579
156092,11.425393,"[2.771110280880264, 2.8150890968323985]",3.950162,NaN,NaN,NaN,2.892386
156093,11.511839,"[-3.874688174564069, -3.864521892821776]",NaN,NaN,NaN,NaN,NaN
156094,10.778310,"[-2.1862591593485767, -2.2374461405723562]",NaN,NaN,NaN,NaN,NaN


In [34]:
pd.pivot_table(df, index = ['overlap'], columns = ['potential_conflict_type'], values = 'pair', aggfunc = 'count')

potential_conflict_type,angled,rear_end,side_swipe
overlap,,,
overlap,1,32160,41081


In [35]:
df[pd.isna(df['overlap']) == False]

,pair,veh_id,LV_ID,frm,LV_type,local_x,local_y,V_len,V_wid,velocity,...,TIT,TIT2,TIACT,TIPICUD,TIDSS,TIDRAC,TIMDRAC,TITAdv,TITimeGap,TIDCIA
2,1000_994,1000,994,17460,LVL,173.125,21.406250,7.09375,2.064453,104.1250,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.000000,0.0,NaN
3,1000_994,1000,994,17457,LVL,170.250,21.359375,7.09375,2.064453,103.8750,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.0,NaN
4,1000_994,1000,994,17454,LVL,167.375,21.328125,7.09375,2.064453,104.3750,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.000000,0.0,NaN
5,1000_994,1000,994,17451,LVL,164.500,21.296875,7.09375,2.064453,103.1875,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.069636,0.0,NaN
8,1000_994,1000,994,17445,LVL,158.750,21.218750,7.09375,2.064453,103.3750,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.000000,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156087,9_6,9,6,747,LVR,168.750,4.464844,1.93457,2.064453,111.8750,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.0,NaN
156088,9_6,9,6,741,LVR,162.625,4.265625,1.93457,2.064453,111.2500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.0,NaN
156090,9_6,9,6,735,LVR,156.500,4.101562,1.93457,2.064453,110.6250,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.0,NaN
156093,9_6,9,6,726,LVR,147.250,3.832031,1.93457,2.064453,109.1250,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.026977,0.0,NaN
